In [1]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import json
from datasets import Dataset, load_dataset

model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

2025-09-30 10:14:04.078958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759227244.417365      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759227244.506833      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
max_length = 384 
doc_stride = 128

In [3]:
with open("/kaggle/input/bert-dataset-qa/train-v0.1.json", "r") as f:
    data = json.load(f)

records = []
for entry in data["data"]:
    for paragraph in entry["paragraphs"]:
        context = paragraph["context"]
        for qa in paragraph["qas"]:
            question = qa["question"]
            qid = qa["id"]
            answers = qa["answers"] 
            for ans in answers:
                records.append({
                    "id": qid,
                    "question": question,
                    "context": context,
                    "answer_text": ans["text"],
                    "answer_start": int(ans["answer_start"])
                })

dataset = Dataset.from_list(records)


In [4]:
dataset[0]

{'id': 1,
 'question': 'el-Hasan b. Muhammed el-Vezzan isimli bilgin avrupa’da nasıl tanınmaktadır ?',
 'context': 'İslam dünyasında bilimin 16. yüzyılda hala yüksek seviyede bulunduğunu gösteren çok ilginç bir örneği deskriptif coğrafya ekolünden verebiliriz. Bize bu örneği, Avrupa’da Afrikalı Leo (Leo Africanus) olarak tanınan el-hasan b. Muhammed el-Vezzan (doğumu yaklaşık 888/1483)’dır. Fas (Fez) şehrinde büyümüş ve eğitimini almış olan Granada doğumlu bu bilgin, diplomatik hizmetler yoluyla, özellikle kuzey Afrika’da olmak üzere birçok İslam ülkesini tanıyıp bir yazar olarak coğrafya ve kartografya ile ilgileniyordu. İstanbul’dan dönüş yolculuğunda Sicilyalı korsanların eline esir düşmüş, ilk olarak Napoli’ye daha sonra Roma’ya satılıp Papa X. Leo tarafından 6.1.1520 yılında bizzat Papa’nın adıyla Giovanni Leo olarak vaftiz edilmişti. İtalya’daki ikameti sırasında İtalyanca öğrendi ve Arapça öğretti. Yazar olarak faaliyetlerini Roma ve Bologna’da devam ettirdi. Afrika coğrafyası d

In [5]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)

        sample_index = sample_mapping[i]
        answer = examples["answer_text"][sample_index]
        start_char = int(examples["answer_start"][sample_index])
        end_char = start_char + len(answer)

        # context kısmının token sınırlarını bul
        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
            token_start_index += 1

        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != 1:
            token_end_index -= 1

        # Eğer cevap context dışında kalıyorsa → CLS’ye yönlendir
        if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            # cevap span’ini token’lara hizala
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                token_start_index += 1
            start_positions.append(token_start_index - 1)

            while offsets[token_end_index][1] >= end_char:
                token_end_index -= 1
            end_positions.append(token_end_index + 1)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions

    return tokenized_examples


In [6]:
tokenized_dataset = dataset.map(prepare_train_features, batched=True, remove_columns=dataset.column_names)

Map:   0%|          | 0/8308 [00:00<?, ? examples/s]

In [7]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./bert-qa",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=100,          
    disable_tqdm = False,
    report_to="none"           # wandb vb. loglama servislerini kapat
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset.select(range(100)),  # küçük bir parça eval için
    tokenizer=tokenizer
)

trainer.train()


/tmp/ipykernel_36/1574120214.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
100,3.279500
200,1.798300
300,1.568000
400,1.308200
500,1.201900
600,1.175300
700,1.073500
800,0.952800
900,0.925700
1000,0.961300


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=1020, training_loss=1.4142309394537234, metrics={'train_runtime': 1405.6744, 'train_samples_per_second': 23.216, 'train_steps_per_second': 0.726, 'total_flos': 6395368769491968.0, 'train_loss': 1.4142309394537234, 'epoch': 3.0})

In [8]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

result = qa_pipeline({
    "context": "Türkiye'nin başkenti tarihsel olarak bir dönem İstanbul olsa da şu anki başkenti Ankara'dır.",
    "question": "Türkiye'nin başkenti neresidir?"
})

print(result)


Device set to use cuda:0


{'score': 0.20872120559215546, 'start': 81, 'end': 87, 'answer': 'Ankara'}


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


In [18]:
import torch
text = r""" 
Ankara'nın başkent ilan edilmesinin ardından (13 Ekim 1923) şehir hızla gelişmiş ve Türkiye'nin ikinci en kalabalık ili olmuştur. 
Türkiye Cumhuriyeti'nin ilk yıllarında ekonomisi tarım ve hayvancılığa dayanan ilin topraklarının yarısı hâlâ tarım amaçlı kullanılmaktadır. 
Ekonomik etkinlik büyük oranda ticaret ve sanayiye dayalıdır. Tarım ve hayvancılığın ağırlığı ise giderek azalmaktadır. 
Ankara ve civarındaki gerek kamu sektörü gerek özel sektör yatırımları, başka illerden büyük bir nüfus göçünü teşvik etmiştir. 
Cumhuriyetin kuruluşundan günümüze, nüfusu ülke nüfusunun iki katı hızda artmıştır. 
Nüfusun yaklaşık dörtte üçü hizmet sektörü olarak tanımlanabilecek memuriyet, ulaşım, haberleşme ve ticaret benzeri işlerde, dörtte biri sanayide, 
%2'si ise tarım alanında çalışır. Sanayi, özellikle tekstil, gıda ve inşaat sektörlerinde yoğunlaşmıştır. Günümüzde ise en çok savunma, metal ve motor sektörlerinde 
yatırım yapılmaktadır. Türkiye'nin en çok sayıda üniversiteye sahip ili olan Ankara'da ayrıca, üniversite diplomalı kişi oranı ülke ortalamasının iki katıdır.
Bu eğitimli nüfus, teknoloji ağırlıklı yatırımların gereksinim duyduğu iş gücünü oluşturur. 
Ankara'dan otoyollar, demir yolu ve hava yoluyla Türkiye'nin diğer şehirlerine ulaşılır. 
Ankara aynı zamanda başkent olarak Türkiye Büyük Millet Meclisi (TBMM)'ye de ev sahipliği yapmaktadır. 
"""

questions = [
    "Ankara kaç yılında başkent oldu?",
    "Ankara ne zaman başkent oldu?",
    "Ankara'dan başka şehirlere nasıl ulaşılır?",
    "TBMM neyin kısaltmasıdır?"
]

for question in questions:
    inputs = tokenizer(question, text, return_tensors="pt", truncation=True)
    inputs = {k: v.to('cuda') for k, v in inputs.items()}  # GPU'ya taşı

    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    input_ids = inputs["input_ids"][0]
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end].cpu())
    )

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")


Question: Ankara kaç yılında başkent oldu?
Answer: 13 Ekim 1923

Question: Ankara ne zaman başkent oldu?
Answer: 13 Ekim 1923 )

Question: Ankara'dan başka şehirlere nasıl ulaşılır?
Answer: otoyollar, demir yolu ve hava yoluyla

Question: TBMM neyin kısaltmasıdır?
Answer: 

